# 그룹 검색
- 그룹화 검색을 사용하면 Milvus는 지정된 필드의 값에 따라 검색 결과를 그룹화하여 더 높은 수준에서 데이터를 집계할 수 있음
- 예를 들어, 기본 ANN 검색을 사용하여 현재 책과 유사한 책을 찾을 수 있지만, 그룹화 검색을 사용하여 해당 책에서 논의되는 주제와 관련된 책 카테고리를 찾을 수 있음

참고자료: https://milvus.io/docs/ko/grouping-search.md

## 개요
- 검색 결과의 엔티티가 스칼라 필드에서 동일한 값을 공유하면 특정 속성이 유사하다는 것을 나타내며, 이는 검색 결과에 부정적인 영향을 미칠 수 있음
- 컬렉션에 여러 문서가 저장되어 있다고 가정
- 문서를 벡터로 변환할 때 가능한 한 많은 의미론적 정보를 유지하기 위해 각 문서는 관리하기 쉬운 작은 단락(또는 청크)로 분할되어 별도의 엔티티로 저장
- 문서가 더 작은 섹션으로 나뉘어 있더라도 사용자는 여전히 자신의 요구와 가장 관련성이 높은 문서를 식별하는 데 관심이 있는 경우가 많음

- 검색 결과의 다양성을 높이기 위해 검색 요청에 group_by_field 매개변수를 추가하여 그룹화 검색을 활성화할 수 있음
- 다이어그램에 표시된 것처럼 group_by_field을 docId로 설정 가능
- 위의 요청을 받으면 Milvus는 다음과 같이 행동
    - 제공된 쿼리 벡터를 기반으로 ANN 검색을 수행하여 쿼리와 가장 유사한 모든 엔티티를 찾음
    - 검색 결과를 지정된 group_by_field에 따라 그룹화함(예: docId)
    - limit 매개변수에 정의된 대로 각 그룹에 대해 각 그룹에서 가장 유사한 엔티티가 있는 상위 결과를 반환

## 그룹 검색 수행


In [1]:
data = [
        {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "chunk": "pink_8682", "docId": 1},
        {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "chunk": "red_7025", "docId": 5},
        {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "chunk": "orange_6781", "docId": 2},
        {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "chunk": "pink_9298", "docId": 3},
        {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "chunk": "red_4794", "docId": 3},
        {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "chunk": "yellow_4222", "docId": 4},
        {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "chunk": "red_9392", "docId": 1},
        {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "chunk": "grey_8510", "docId": 2},
        {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "chunk": "white_9381", "docId": 5},
        {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "chunk": "purple_4976", "docId": 3},
]

In [3]:
from pymilvus import MilvusClient, DataType

# 클라이언트 연결
client = MilvusClient(uri="http://localhost:19530")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

# 정확하게 정의된 스키마
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=False)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="color", datatype=DataType.VARCHAR, max_length=32)
schema.add_field(field_name="likes", datatype=DataType.INT64)
schema.add_field(field_name="docId", datatype=DataType.INT64)

index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="IP")

# 컬렉션 생성
client.create_collection(
    collection_name="my_collection",
    vector_field_name="vector",
    schema=schema,
    index_params=index_params
)

In [4]:
query_vectors = [
    [0.14529211512077012, 0.9147257273453546, 0.7965055218724449, 0.7009258593102812, 0.5605206522382088]]

# Group search results
res = client.search(
    collection_name="my_collection",
    data=query_vectors,
    limit=3,
    group_by_field="docId",
    output_fields=["docId"]
)

# Retrieve the values in the `docId` column
doc_ids = [result['entity']['docId'] for result in res[0]]

## 그룹 크기 구성
- 기본적으로 그룹 검색은 그룹당 하나의 엔티티만 반환
- 그룹당 여러 개의 결과를 원하면 group_size 및 strict_group_size 매개변수를 조정

In [7]:
res = client.search(
    collection_name="my_collection", 
    data=query_vectors, # query vector
    limit=5, # number of groups to return
    group_by_field="docId", # grouping field
    group_size=2, # p to 2 entities to return from each group
    strict_group_size=True, # return exact 2 entities from each group
    output_fields=["docId"]
)


In [8]:
print(res)

data: [[]]
